# Data exploration and validation

In this exercise we will cover how to use Polars and Pandera to explore, tidy, and validate the data.

## Task 1 - load data from Pin

### 🔄 Task

- Use `polars` to load the data from Posit Connect into a Polars dataframe.

### 🧑‍💻 Code

In [ ]:
import os
from pathlib import Path

import polars as pl
from dotenv import load_dotenv
import pins

In [ ]:
# Get the API key and server URL from an environment variable.
if Path(".env").exists():
    load_dotenv()

connect_server = os.environ["CONNECT_SERVER"]
connect_api_key = os.environ["CONNECT_API_KEY"]

In [ ]:
# Set up a pins board.
board = pins.board_connect(server_url=connect_server, api_key=connect_api_key)
board

In [ ]:
# Update the username with your Posit Connect username.
username = "sam.edwardes"

Get the vessel verbose data set.

In [ ]:
vessel_verbose_paths = board.pin_download(f"{username}/vessel_verbose_raw")
vessel_verbose_paths

In [ ]:
vessel_verbose = pl.read_parquet(vessel_verbose_paths)
vessel_verbose

Get the vessel verbose history data set.

In [ ]:
vessel_history_paths = board.pin_download(f"{username}/vessel_history_raw")
vessel_history_paths

In [ ]:
vessel_history = pl.read_parquet(vessel_history_paths)
vessel_history

## Task 2 - explore the data

### 🔄 Task

Begin exploring the data. You will want to understand.

- What columns exist in the data?
- How do the two data sets relate to one another?
- What is the type of each column (e.g. string, number, category, date)?
- Which columns could be useful for the model.
- What is the cardinality of categorical data?
- Is all of the data in scope?
- What steps will I need to perform to clean the data?

**Tips**

- Use VS Codes built in data viewer to explore the data.
- If you are more comfortable with Pandas, you can convert the polars dataframe into a pandas dataframe (e.g. `df.to_pandas()`).
- The polars user guide has great docs on how to use polars: https://docs.pola.rs.

🚨 We are not performing feature engineering at this stage. But it is a good time to start thinking about what features you can create from the data.

> 💡 We are not using it in this workshop, but `ydata-profiling` (<https://github.com/ydataai/ydata-profiling>) is a good tool for exploring a new dataset.

### 🧑‍💻 Code

#### vessel_history

In [ ]:
(
    vessel_history
    .head(3)
)

- The dates and times are not formatted correctly. We can fix this when we tidy the data.

#### vessel_verbose

In [ ]:
(
    vessel_verbose
    .head(2)
)

How many different vessels are in the data?

In [ ]:
# Print more rows.
pl.Config.set_tbl_rows(100)

In [ ]:
(
    vessel_verbose
    .select(pl.col('VesselID'), pl.col('VesselName'))
)

In [ ]:
# Verify that each VesselID is unique.
(
    vessel_verbose
    .get_column('VesselID')
    .n_unique()
)

What are all of the numerical columns?

In [ ]:

(
    vessel_verbose
    .select(pl.selectors.numeric())
    .head(5)
)

- Some of the date based columns are integers or floats. During data tidying we could convert them into a proper date type.


What are all of the string columns?

In [ ]:
(
    vessel_verbose
    .select(pl.selectors.string())
    .head(5)
)

- It looks like some missing values are represented with an empty string `""` while others have a `null` value. We may want to make this consistent when we tidy the data.
- Some string columns are measurements that should be converted into numeric types.

How much data is missing?

In [ ]:
(
    vessel_verbose
    .null_count()
    .transpose(include_header=True)
    .rename({"column": "Column Name", "column_0": "Missing Rows"})
    .with_columns(((pl.col("Missing Rows") / vessel_verbose.shape[0]) * 100).round(1).alias('% Missing'))
    .sort("Missing Rows", descending=True)
)

Whats in the `Class` column?

In [ ]:
(
    vessel_verbose
    .get_column("Class")
    .head(2)
)

The class column contains a `struct`: https://docs.pola.rs/user-guide/expressions/structs/

> Polars `Structs` are the idiomatic way of working with multiple columns. It is also a free operation i.e. moving columns into Structs does not copy any data!

Lets look more closely at the `Class` column for Cathlamet.

In [ ]:
(
    vessel_verbose
    .filter(pl.col("VesselName") == "Cathlamet")
    .get_column("Class")
    .to_list()
)

It looks like the `Class` column contains a list with a single dictionary. When we tidy this data we can make it easier to work with by unnesting this data and moving it into its own columns.

## Task 3 - Tidy the Data

### 🔄 Task

Now that you have a basic understanding of the data, the next step is to tidy the data.

### 🧑‍💻 Code

#### vessel_history

In [ ]:
vessel_history.head(2)

Convert the datetimes from strings to polars datetime objects. The logic is pretty complex. So we will abstract it into a function that we can apply to all of the required columns.

In [ ]:
def convert_string_to_datetime(series: pl.Series) -> pl.Series:
    """
    Convert the datetime format from wadot into a datetime format that polars
    can understand.

    >>> convert_string_to_datetime(pl.Series(['/Date(1714547700000-0700)/']))
    shape: (1,)
    Series: '' [datetime[μs, UTC]]
    [
        2024-05-01 07:15:00 UTC
    ]
    """
    # Extract the unix time stamp. To work with polars we need the time
    # the number of seconds since 1970-01-01 00:00 UTC, so divide by
    # 1_000.
    unix_timestamp = (
        (series.str.extract(r"/Date\((\d{13})[-+]").cast(pl.Int64) / 1_000)
        .cast(pl.Int64)
        .cast(pl.String)
    )
    # Extract the timezone.
    timezone = series.str.extract(r"([-+]\d{4})")
    # Create a new series that has the timestamp and timezone.
    clean_timestamp = unix_timestamp + timezone
    # Convert into a datetime.
    datetime_series = clean_timestamp.str.to_datetime("%s%z")
    return datetime_series


convert_string_to_datetime(pl.Series(['/Date(1714547700000-0700)/']))

In [ ]:
vessel_history_clean = (
    vessel_history
    .with_columns(
        (
            pl
            .col("ScheduledDepart", "ActualDepart", "EstArrival", "Date")
            .map_batches(lambda s: convert_string_to_datetime(s))
        )
    )
)

In [ ]:
vessel_history_clean.head(5)

Normalize all of the string columns so that they are consistent.

In [ ]:
vessel_history_clean = (
    vessel_history_clean
    .with_columns(
        (
            pl
            .col("Vessel", "Departing", "Arriving")
            .str.to_lowercase()
            .str.strip()
        )
    )
)

In [ ]:
vessel_history_clean.head(5)

#### vessel_verbose

In [ ]:
vessel_verbose.head(3)

Convert the length measurements into a numeric value. Again we will use a function to capture this complex logic.

In [ ]:
def convert_measurement_string_to_feet(series: pl.Series) -> pl.Series:
    """
    Convert the measurement string into a float.
    """
    feet = series.str.extract(r"(\d+)'").cast(pl.Int32)
    inches = series.str.extract(r'(\d+)"').cast(pl.Int32).fill_null(0)
    total_inches = feet * 12 + inches
    return total_inches


convert_measurement_string_to_feet(pl.Series(['''78' 8"''', """64'""", '''100' 11"''']))

In [ ]:
vessel_verbose_clean = (
    vessel_verbose
    .with_columns(
        pl.col("Beam", "Length", "Draft").map_batches(lambda s: convert_measurement_string_to_feet(s)).name.suffix("Inches"),
    )
    .select(pl.col("*").exclude(["Beam", "Length", "Draft"]))
)

In [ ]:
vessel_verbose_clean.head(5)

Fix the dates.

In [ ]:
vessel_verbose_clean = (
    vessel_verbose_clean
    .with_columns(
        pl.col("YearBuilt").cast(pl.String).str.to_date("%Y"),
        pl.col("YearRebuilt").cast(pl.Int32).cast(pl.String).str.to_date("%Y"),
    )
)

In [ ]:
vessel_verbose_clean.head(5)

Fix numeric types.

In [ ]:
vessel_verbose_clean = (
    vessel_verbose_clean
    .with_columns(
        pl.col("MaxPassengerCountForInternational").cast(pl.Int32),
    )
)

In [ ]:
vessel_verbose_clean.head(5)

The column `Class` is a struct. Each row contains a dictionary object of key value pairs.

In [ ]:
vessel_verbose_clean.get_column('Class')

In [ ]:
vessel_verbose_clean.get_column('Class').to_list()[0]

This data would be easier to work with if it was in a tabular format, and not a nested dictionary. To do this, unnest the `Class` struct so that each data point is in its own column.

In [ ]:
vessel_verbose_clean = (
    vessel_verbose_clean
    .unnest("Class")
)

In [ ]:
vessel_verbose_clean.head(2)

Handle missing values for strings.

In [ ]:
vessel_verbose_clean = (
    vessel_verbose_clean
    .with_columns(
        pl.col(pl.String).replace(" ", None),
    )
)

In [ ]:
vessel_verbose_clean.head(2)

Normalize all of the string columns so that they are consistent.

In [ ]:
vessel_verbose_clean = (
    vessel_verbose_clean
    .with_columns(
        (
            pl
            .col("VesselName", "VesselAbbrev", "ClassName", "CityBuilt", "PropulsionInfo")
            .str.to_lowercase()
            .str.strip()
        )
    )
)

In [ ]:
vessel_verbose_clean.head(5)

Drop the VesselDrawingImg column, it does not contain an data.

In [ ]:
vessel_verbose_clean.get_column('VesselDrawingImg')

In [ ]:
vessel_verbose_clean = (
    vessel_verbose_clean
    .drop('VesselDrawingImg')
)

In [ ]:
vessel_verbose_clean.head(5)

## Task 4 - Validate the Data

### 🔄 Task

In the previous activity we tidied the dataset. For some projects, this may be enough. However, for this project we plan to refresh the data on a regular basis. We would like to gain additional comfort that the data we are using is correct. Data validation can help prove that our data tidying was correct, and find any potential issues if the upstream data changes.

[Pandera](https://pandera.readthedocs.io/en/stable/) is a Python library for validating Pandas dataframes. There are two steps:

1. Define a schema for your data. For example:
   - Define the type for each column
   - Confirm if null values are allowed
   - Define custom checks
2. Run your data through the schema validator.

You will find these links useful when defining your schema:

- Polars data validation guide: https://pandera.readthedocs.io/en/stable/polars.html#usage
- Polars data types: https://pandera.readthedocs.io/en/stable/reference/dtypes.html#polars-dtypes
- `pa.Field` API: https://pandera.readthedocs.io/en/stable/reference/generated/pandera.api.dataframe.model_components.Field.html#pandera.api.dataframe.model_components.Field
- List of built in checks you can use with `pa.Field`: https://pandera.readthedocs.io/en/stable/reference/generated/pandera.api.checks.Check.html#pandera.api.checks.Check

### 🧑‍💻 Code

#### vessel_history

Start by validating the vessel_history data set. As a reminder, here is what the data looks like:

In [ ]:
vessel_history_clean.head(3)

The class below defines the schema and checks for the vessel_history data set.

- Each column is a class attribute. At a minimum, we define the column type (e.g. int, str, datetime, etc.)
- For some columns, we use `pa.Field` to add more checks. For example in the EstArrival column we are going to allow nullable values.
- We can define additional and more complex column and dataframe level checks by defining class methods.

In [ ]:
import pandera.polars as pa
from pandera.engines.polars_engine import DateTime, Date, Int32


class VesselHistorySchema(pa.DataFrameModel):
    VesselId: int
    Vessel: str
    Departing: str
    Arriving: str = pa.Field(nullable=True)
    ScheduledDepart: DateTime = pa.Field(dtype_kwargs={"time_zone": "UTC"})
    ActualDepart: DateTime = pa.Field(dtype_kwargs={"time_zone": "UTC"})
    EstArrival: DateTime = pa.Field(dtype_kwargs={"time_zone": "UTC"}, nullable=True)
    Date: DateTime = pa.Field(
        dtype_kwargs={"time_zone": "UTC"},
        ge=pl.datetime(2020, 1, 1, time_zone="America/Vancouver").dt.convert_time_zone("UTC")
    )

    @pa.dataframe_check
    def year_of_date_matches_scheduled_depart(cls, df: pa.PolarsData) -> pl.LazyFrame:
        """
        Verify that the year of the Date column matches the year of the
        ScheduledDepart column.
        """
        return df.lazyframe.select(pl.col("Date").dt.year().eq(pl.col("ScheduledDepart").dt.year()))

    @pa.dataframe_check(raise_warning=True)
    def estimated_arrival_is_after_scheduled_depart(cls, df: pa.PolarsData) -> pl.LazyFrame:
        """
        Verify that the EstArrival date time is always after the ScheduledDepart
        date time.

        Note this check is expected to fail, therefore raise_warning=True is
        used. In the future we should go back and understand why this check
        fails.
        """
        return df.lazyframe.select(pl.col("EstArrival").ge(pl.col("ScheduledDepart")))

    @pa.check("VesselId", raise_warning=True)
    def vessel_ids_in_vessel_verbose_data_set(cls, data: pa.PolarsData) -> pl.LazyFrame:
        """
        Verify that all of the vessels in the vessel history data set also exist
        in the vessel verbose data set.

        Note this check is expected to fail, therefore raise_warning=True is
        used. In the future we should go back and understand why this check
        fails.
        """
        vessel_ids = vessel_verbose_clean.get_column("VesselID").to_list()
        return data.lazyframe.select(pl.col(data.key).is_in(vessel_ids))


To validate the data, run the dataframe through the `pa.DataFrameModel.validate` method.

In [ ]:
vessel_history_validated = VesselHistorySchema.validate(vessel_history_clean)

In [ ]:
vessel_history_validated.head()

- Are there any more checks that you would add?
- How should we handle the data that fails the two checks that raise a warning instead of fail?
- Try changing some of the validations so that they fail? Are you able to use the failure message to identify the bad data?

#### vessel_verbose

In the interest of time, we will "skim" over the validation of the vessel_verbose data set. The class below defines the schema and checks for the vessel_verbose data set.

*💁 Note: time permitting walk the learners through using multiple cursors and split editors in VS Code and how they can be used to quickly create the code for the DataFrame model.*

```python
vessel_verbose_clean.columns
vessel_verbose_clean.head(2).transpose(include_header=True)
```



In [ ]:
vessel_verbose_clean.head(2)

In [ ]:
class VesselVerboseSchema(pa.DataFrameModel):
    VesselID: int
    VesselSubjectID: int
    VesselName: str
    VesselAbbrev: str
    ClassID: int
    ClassName: str
    ClassSubjectID: int
    DrawingImg: str
    PublicDisplayName: str
    SilhouetteImg: str
    SortSeq: int
    Status: int
    OwnedByWSF: bool
    CarDeckRestroom: bool
    CarDeckShelter: bool
    Elevator: bool
    ADAAccessible: bool
    MainCabinGalley: bool
    MainCabinRestroom: bool
    PublicWifi: bool
    ADAInfo: str
    AdditionalInfo: str = pa.Field(nullable=True)
    VesselNameDesc: str
    VesselHistory: str = pa.Field(nullable=True)
    CityBuilt: str
    SpeedInKnots: int
    EngineCount: int
    Horsepower: int
    MaxPassengerCount: int
    PassengerOnly: bool
    FastFerry: bool
    PropulsionInfo: str
    TallDeckClearance: int
    RegDeckSpace: int
    TallDeckSpace: int
    Tonnage: int
    Displacement: int
    YearBuilt: Date
    YearRebuilt: Date = pa.Field(nullable=True)
    SolasCertified: bool
    MaxPassengerCountForInternational: Int32 = pa.Field(nullable=True)
    BeamInches: Int32
    LengthInches: Int32
    DraftInches: Int32 = pa.Field(nullable=True)

    @pa.check('DrawingImg')
    def validate_urls(cls, data: pa.PolarsData) -> pl.LazyFrame:
        return (
            data
            .lazyframe
            .select(pl.col(data.key).str.starts_with("https://"))
        )

In [ ]:
vessel_verbose_validated = VesselVerboseSchema.validate(vessel_verbose_clean)

In [ ]:
vessel_verbose_validated.head(5)

## Task 5 - Publish data to Posit Connect

### 🔄 Task

Save the clean data to Posit Connect as a pin.

### 🧑‍💻 Code

#### vessel_history

In [ ]:
board.pin_write(
    vessel_history_clean.to_pandas(), f"{username}/vessel_history_clean", type="parquet"
)

#### vessel_verbose

In [ ]:
board.pin_write(
    vessel_verbose_clean.to_pandas(), f"{username}/vessel_verbose_clean", type="parquet"
)

## Task 6 - publish notebook as Quarto document to Posit Connect

### 🔄 Task

### 🧑‍💻 Code